In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

class SimpleNet(torch.nn.Module):
    # TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        # self.hidden = torch.nn.Linear(784,548)   # hidden layer
        # self.out = torch.nn.Linear(548,10)   # output layer
        self.fc1 = nn.Linear(784, 576)
        self.bc1 = nn.BatchNorm1d(576)

        self.fc2 = nn.Linear(576, 324)
        self.bc2 = nn.BatchNorm1d(324)

        self.fc3 = nn.Linear(324, 144)
        self.bc3 = nn.BatchNorm1d(144)

        self.fc4 = nn.Linear(144, 10)

    def forward(self, x):
        # x = F.relu(self.hidden(x))      # activation function for hidden layer
        # x = self.out(x)
        # return x
        x = x.view((-1, 784))
        h = self.fc1(x)
        h = self.bc1(h)
        h = nn.functional.relu(h)
        h = nn.functional.dropout(h, p=0.5, training=self.training)  

        h = self.fc2(h)
        h = self.bc2(h)
        h = nn.functional.relu(h)
        h = nn.functional.dropout(h, p=0.2, training=self.training)  

        h = self.fc3(h)
        h = self.bc3(h)
        h = nn.functional.relu(h)
        h = nn.functional.dropout(h, p=0.1, training=self.training)  

        h = self.fc4(h)
        out = nn.functional.log_softmax(h, dim=0)
        return out

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    train_acc = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()  
        out = model(images)  
        lossvalue = criterion(out, labels) 
        optimizer.zero_grad()  
        lossvalue.backward()  
        optimizer.step() 

        # 计算损失
        train_loss += float(lossvalue)
        # 计算精确度
        _, pred = out.max(1)
        num_correct = (pred == labels).sum()
        acc = int(num_correct) / images.shape[0]
        train_acc += acc
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    eval_loss = 0
    eval_acc = 0
    model.eval()  
    for images, labels in tqdm(test_loader):
        images = images.view(-1, 784)
        testout = model(images)
        testloss = criterion(testout, labels)
        eval_loss += float(testloss)

        _, pred = testout.max(1)
        num_correct = (pred == labels).sum()
        acc = int(num_correct) / images.shape[0]
        eval_acc += acc
        
    train_loss = train_loss / len(train_loader)
    train_acc = train_acc / len(train_loader)
    eval_loss = eval_loss / len(test_loader)
    eval_acc = eval_acc / len(test_loader)
    print("[Epoch: %d] Train Loss: %5.5f Train Accuracy: %5.5f" % (epoch + 1, train_loss, train_acc))
    print("[Epoch: %d] Test Loss: %5.5f Test Accuracy: %5.5f" % (epoch + 1, eval_loss, eval_acc))

print('Training Accuracy: %.2f%%' % (train_acc * 100))
print('Testing Accuracy: %.2f%%' % (eval_acc * 100))

  1%|          | 3/468 [00:00<00:16, 28.22it/s]

[Epoch: 1] Train Loss: 0.78177 Train Accuracy: 0.81916
[Epoch: 1] Test Loss: 0.26786 Test Accuracy: 0.93640


  1%|          | 3/468 [00:00<00:18, 25.57it/s]

[Epoch: 2] Train Loss: 0.20573 Train Accuracy: 0.93919
[Epoch: 2] Test Loss: 0.14152 Test Accuracy: 0.95613


  1%|          | 3/468 [00:00<00:17, 26.20it/s]

...

[Epoch: 8] Train Loss: 0.04488 Train Accuracy: 0.98533
[Epoch: 8] Test Loss: 0.07410 Test Accuracy: 0.97796


  1%|          | 3/468 [00:00<00:17, 26.62it/s]

[Epoch: 9] Train Loss: 0.04136 Train Accuracy: 0.98683
[Epoch: 9] Test Loss: 0.07132 Test Accuracy: 0.97786


100%|██████████| 78/78 [00:01<00:00, 51.27it/s]

[Epoch: 10] Train Loss: 0.03562 Train Accuracy: 0.98835
[Epoch: 10] Test Loss: 0.06808 Test Accuracy: 0.97827
Training Accuracy: 98.83%
Testing Accuracy: 97.83%
